## 1. Packages

In [1]:
using Agents
using GLMakie
using Random
using LinearAlgebra
using StaticArrays
using Distributions
using OrderedCollections: OrderedDict

## 2. Agent definition

In [ ]:
@agent struct Agent(ContinuousAgent{2,Float64}) # Agent in 2D continuous space
    infected::Bool
    recovered::Bool
    transmission_rate::Float64
    recovery_rate::Float64
    speed::Float64
    scale::Float64
end

## 3. Helper - random unit vector



In [ ]:
# Random unit velocity in 2D.
random_unit_vec() = begin
    v = rand(2) .- 0.5 # Random vector 
    v ./= max(norm(v)) # Normalize to unit vector
    v = SVector{2,Float64}(v) # Convert to static vector for speed
end

random_unit_vec (generic function with 1 method)

## 4. Dynamics - agent_step! - model_step!



In [ ]:
function agent_step!(a, model)
    dm = model.disease_model_index
    susceptible = !a.infected && !(dm == 3 && a.recovered) # Susceptible if not infected and (not recovered if SIR)
    if susceptible
        k = 0 # Count of infected neighbors
        for n in nearby_agents(a, model, 1.0) # Count infected neighbors within radius 1
            if n.infected == true
                k += 1 # Increment count of infected neighbors
            end
        end
        p_inf = 1 - ((1 - model.transmission_rate)^k) * (1 - model.spontaneous_infect) # Probability of infection from neighbors and spontaneous infection
        if rand() < p_inf
            a.infected = true
        end
    end

    # move agent 
    θmax = deg2rad(model.turning_angle) # Maximum turning angle in radians
    δθ = (rand() - rand()) * θmax       # Random angle 
    ϕ = atan(a.vel[2], a.vel[1]) + δθ   # Current direction + angle change
    a.vel = @SVector [cos(ϕ), sin(ϕ)]   # New velocity vector
    move_agent!(a, model, model.speed)
end

agent_step! (generic function with 1 method)

In [ ]:
function model_step!(model)
    dm = model.disease_model_index
    infected_start = Set(a.id for a in allagents(model) if a.infected) # IDs of currently infected agents

    if dm == 2  # SIS
        for id in infected_start
            a = model[id]
            if a.infected && rand() < model.recovery_rate # Chance of recovery back to susceptible
                a.infected = false
                a.recovered = false
            end
        end
    elseif dm == 3 # SIR 
        for id in infected_start
            a = model[id]
            if a.infected && rand() < model.recovery_rate # Chance of recovery to recovered state
                a.infected = false
                a.recovered = true
            end
        end
    end

    inf_frac = count(a -> a.infected, allagents(model)) / nagents(model) # Calculate fraction of infected agents

    if inf_frac > model.max_infected_fraction # Update max infected fraction if current fraction exceeds it
        model.max_infected_fraction = inf_frac
    end
end


model_step! (generic function with 1 method)

## 5. Model initialization

In [6]:
function initialize_model(; n_agents::Int=500,
    transmission_rate::Float64=0.10,
    recovery_rate::Float64=0.01,
    spontaneous_infect::Float64=0.0,
    speed::Float64=1.0,
    scale::Float64=0.7,
    extent::Tuple{<:Real,<:Real}=(100, 100), # Size of the 2D space
    turning_angle::Float64=360.0,
    disease_model::Symbol=:SI,               # :SI, :SIS, :SIR
    initial_infected::Int=1)

    space = ContinuousSpace(extent; spacing=1, periodic=true)

    model = StandardABM(
        Agent, space;
        properties=Dict(
            :turning_angle => turning_angle,
            :disease_model_index => (disease_model === :SI ? 1 : disease_model === :SIS ? 2 : 3), # Map model symbol to index
            :max_infected_fraction => 0.0,
            :speed => speed,
            :transmission_rate => transmission_rate,
            :recovery_rate => recovery_rate,
            :spontaneous_infect => spontaneous_infect,
        ),
        agent_step!, model_step!,
        scheduler=Schedulers.Randomly()
    )
    # Add agents with random positions and initial infection state
    for i in 1:n_agents
        pos = (rand() * extent[1], rand() * extent[2]) #
        infected = i ≤ initial_infected
        recovered = false
        a = add_agent!(model, pos, infected, recovered,
            transmission_rate, recovery_rate, speed, scale)
        a.vel = random_unit_vec()
    end
    return model
end

initialize_model (generic function with 1 method)

## 6. GUI setup

In [ ]:
# Red for infected, blue for recovered, black for susceptible
agent_color(a::Agent) = a.infected ? :red : (a.recovered ? :blue : :black)

# Time series to show: S/I/R fractions & running max infected fraction
mdata = [
    m -> count(a -> !a.infected && !a.recovered, allagents(m)) / nagents(m),
    m -> count(a -> a.infected, allagents(m)) / nagents(m),
    m -> count(a -> a.recovered, allagents(m)) / nagents(m),
    m -> m.max_infected_fraction,
]
mlabels = ["Susceptible fraction", "Infected fraction", "Recovered fraction", "Max infected fraction"]

# Sliders in GUI
params = OrderedDict(
    :disease_model_index => 1:1:3, # 1=SI, 2=SIS, 3=SIR
    :speed => 0.1:0.1:5.0,
    :transmission_rate => 0.0:0.01:1.0,
    :recovery_rate => 0.0:0.01:1.0,
    :spontaneous_infect => 0.0:0.01:1.0,
    :turning_angle => 0.0:15.0:360.0,
);


## 7. GUI visualization helper

In [8]:

# Triangle marker rotated by velocity direction.
function particle_marker(b::Agent)
    scale_factor = 1 * b.scale
    particle_polygon = Makie.Polygon(Point2f[ # Custom shape
        (-scale_factor, -scale_factor),
        (2 * scale_factor, 0),
        (-scale_factor, scale_factor)
    ])
    φ = atan(b.vel[2], b.vel[1])
    rotate_polygon(particle_polygon, φ)
end

particle_marker (generic function with 1 method)

## 8. GUI & controls

In [ ]:
# Run GUI 
fig, abmobs = abmexploration(
    initialize_model(n_agents=500, disease_model=:SI); # n_agents PARAM 
    agent_color=agent_color,
    agent_marker=particle_marker,
    mdata=mdata,
    mlabels=mlabels,
    params=params,
    agentsplotkwargs=(; inspectable=false),
    mplotkwargs=(; inspectable=false)
)
display(fig)


GLMakie.Screen(...)